In [ ]:
import logging
import os
import tempfile
from abc import ABC, abstractmethod
from copy import deepcopy
from pathlib import Path
from typing import Type

import numpy as np
from ConfigSpace import Configuration, ConfigurationSpace
from smac import AlgorithmConfigurationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

from src.constant import DATA_DIR, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.configuration_space.LKH import CONFIGURATION_SPACE

In [ ]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [ ]:
t_c = 100
t_v = 100
K = 2
n = 2
solver_class = TSP_LKH_Solver

experiment = GlobalExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
)

experiment.construct_portfolio(train_instances)

In [ ]:
t_c = 100
t_v = 100
K = 2
n = 2
solver_class = TSP_LKH_Solver

experiment = ParhydraExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
)

experiment.construct_portfolio(train_instances)

In [ ]:
from src.database import db_connect
conn = db_connect()

In [ ]:
import pandas as pd
pd.options.display.max_columns = 999

In [ ]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

In [ ]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

In [ ]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")

In [ ]:
df

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index